# 第一章：提示语基本结构

* **理论讲解**
* **实践练习**
* **示例演练区**

## 环境准备

请运行以下初始化单元以加载API密钥，并建立`get_completion`辅助函数。


In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

---
## 课程说明

Anthropic提供两种API接口：传统的**文本补全API**和现行的**消息API**。本教程将全程采用消息API进行教学。

调用Claude消息API至少需要以下参数：

* `model`：指定调用的**模型名称**
* `max_tokens`：生成文本的最大token数限制（注意Claude可能在达到上限前自主停止）。此参数设定的是绝对最大值，且属于*强制中断*机制，可能导致输出在单词或句子中途截断。
* `messages`：输入消息数组。我们的模型基于"用户-助手"轮转对话机制训练。创建新`Message`时需通过此参数传递历史对话记录，模型将据此生成后续对话内容。
  * 每条消息必须是包含`role`和`content`字段的对象。可传入单条用户消息，也可传入交替排列的多条对话记录（必须严格遵循轮替顺序）。首条消息必须为用户角色。

可选参数包括：

* `system`：系统提示词——后续将详细说明
* `temperature`：控制输出随机性的参数。本课程所有练习中该值固定为0。

完整参数列表请参阅**官方API文档**。
---

### 示例

让我们观察Claude对几个标准格式提示的响应。请依次运行以下每个代码单元（使用shift+enter快捷键），Claude的响应将显示在对应代码块下方。

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

现在让我们观察几个不符合消息API格式规范的提示案例。对于这些格式错误的提示，消息API将返回错误信息。

首先展示的是messages数组中缺失role和content字段的API调用示例。

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

这里展示一个未能遵循user与assistant角色交替规则的提示案例。


In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

用户消息与助手消息必须交替出现，且对话必须以用户回合开始。在提示词中可包含多组用户与助手对话回合（模拟多轮对话场景）。您还可以将文本放入终结性助手消息中，让Claude从您中断处继续生成内容（后续章节将详细说明）。

#### 系统提示词

您还可使用系统提示词。系统提示词是一种在"用户"回合提出问题或任务之前，为Claude提供背景信息、操作指南和原则说明的方式。

在结构上，系统提示词独立于用户与助手消息列表之外，因此需置于单独的system参数中（可参考本笔记**设置**章节中get\_completion辅助函数的结构）。在本教程中，所有可能使用系统提示词的地方，我们都在补全函数中为您提供了system字段。若您不需要使用系统提示词，只需将SYSTEM\_PROMPT变量设为空字符串即可。

#### System Prompt Example

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

Why use a system prompt? A **well-written system prompt can improve Claude's performance** in a variety of ways, such as increasing Claude's ability to follow rules and instructions. For more information, visit our documentation on [how to use system prompts](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts) with Claude.

Now we'll dive into some exercises. If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## 练习
- [练习1.1 - 数到三](#exercise-11---counting-to-three)
- [练习1.2 - 系统提示](#exercise-12---system-prompt)

### Exercise 1.1 - Counting to Three
使用适当的用户/助手格式编辑下面的提示，让Claude​​数到三​​。输出还会显示你的解决方案是否正确。

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

### Exercise 1.2 - System Prompt

把`SYSTEM_PROMPT`改成让Claude像三岁小孩那样说话。

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ 想要提示的话，就运行下面这个单元格吧！

In [1]:
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

The grading function in this exercise is looking for answers that contain "soo" or "giggles".
There are many ways to solve this, just by asking!


### Congrats!

若已完成至此所有练习，君可启程下一章。祝君提示愉快！

---

## Example Playground

这里是供你自由尝试本课所示提示示例的空间，可调整提示以观察其对Claude回答的影响。

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))